In [1]:
import random
import os
import copy
os.environ["OMP_NUM_THREADS"] = "1"
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import cv2
import warnings
import timeit
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch import optim
from scipy.special import expit
from skimage import io, transform, measure
from sklearn import metrics
import optuna
from config import *
from util import *
from models import *
sns.set()
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
rc = {"figure.figsize" : (9, 7),
      "axes.spines.left" : False,
      "axes.spines.right" : False,
      "axes.spines.bottom" : False,
      "axes.spines.top" : False,
      "xtick.bottom" : True,
      "xtick.labelbottom" : False,
      "ytick.labelleft" : False,
      "ytick.left" : True,
      "axes.grid" : False}
plt.rcParams.update(rc)

In [2]:
%%time
torch.cuda.empty_cache()
random.seed(seed)

params = {'batch_size': 16, 'char_dropout': 0.687815, 'dx_dropout': 0.673086, 'learning_rate': 0.000045}

params = {'batch_size': 32, 'char_dropout': 0.646470, 'dx_dropout': 0.323233, 'learning_rate': 0.000018}

params = {'batch_size': 32, 'char_dropout': 0.402177, 'dx_dropout': 0.567145, 'learning_rate': 0.000011}
				
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints", save_top_k=1, monitor="val/loss")
model = CharacteristicsClassifier(img_dir=img_dir, annotations_dir=annotations_dir, metadata_file=metadata_file, weighted_sampling=False,
                                  batch_size=params['batch_size'], char_dropout=params['char_dropout'], learning_rate=params['learning_rate'], 
                                  char_pos_weight=char_pos_weight, dx_pos_weight=dx_pos_weight)
trainer = pl.Trainer(max_epochs=25, devices=1, accelerator="gpu", deterministic=True, callbacks=[checkpoint_callback])
trainer.fit(model)


# Load the best checkpoint after training
best_checkpoint_path = checkpoint_callback.best_model_path
checkpoint = torch.load(best_checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

   | Name                 | Type              | Params
------------------------------------------------------------
0  | lossD                | BCEWithLogitsLoss | 0     
1  | lossC                | BCEWithLogitsLoss | 0     
2  | lossA                | DiceLoss          | 0     
3  | base_model           | ResNet            | 23.5 M
4  | diagnosis_head       | Sequential        | 2.0 K 
5  | characteristics_head | Sequential        | 20.5 K
6  | sigmoid              | Sigmoid           | 0     
7  | accuracy             | Accuracy          | 0     
8  | auroc                | AUROC             | 0     
9  | sensitivity          | Recall            | 0     
10 | specificity          | Specificity       | 0     
------------------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

CPU times: user 7min 41s, sys: 2min 7s, total: 9min 49s
Wall time: 10min 26s


<All keys matched successfully>

In [3]:
 # Val predictions
result_val = get_predictions(trainer, model, split='val', char_threshold=0, dx_threshold=0)
char_threshold = get_thresholds(result_val)
dx_threshold = find_optimal_cutoff(result_val['benign_malignant'], result_val['dx_pred'])
model.dx_threshold = dx_threshold
model.char_threshold = char_threshold
print('Validation:')
display_metrics(result_val)

# Test predictions
result_test = get_predictions(trainer, model, split='test', char_threshold=char_threshold, dx_threshold=dx_threshold)
print('Test:')
display_metrics(result_test)


# External predictions
result_external = get_predictions(trainer, model, split='external', char_threshold=char_threshold, dx_threshold=dx_threshold)
print('External:')
display_metrics(result_external)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 0it [00:00, ?it/s]

TypeError: object of type 'NoneType' has no len()

In [4]:
torch.save(model.state_dict(), os.path.join("models", "combined_model.pth"))
result_val.to_csv(os.path.join("results", "combined_model", "result_val.csv"), index=False)
result_test.to_csv(os.path.join("results", "combined_model", "result_test.csv"), index=False)
result_external.to_csv(os.path.join("results", "combined_model", "result_external.csv"), index=False)

In [3]:
display_metrics(result_test[result_test.lesion_id.str.contains('HAM')])

NameError: name 'display_metrics' is not defined

In [6]:
display_scores(result_test[result_test.lesion_id.str.contains('HAM')])


=====
TRBL
AUC: 0.8835926293721725
Balanced Acc: 0.8311817279046674
Sensitivity: 0.8070175438596491
Specificity: 0.8553459119496856
=====


=====
BDG
AUC: 0.8834355828220859
Balanced Acc: 0.806806343326774
Sensitivity: 0.7547169811320755
Specificity: 0.8588957055214724
=====


=====
GP
AUC: 0.8917050691244239
Balanced Acc: 0.8237327188940091
Sensitivity: 0.7903225806451613
Specificity: 0.8571428571428571
=====


=====
WLSA
AUC: 0.8985663082437276
Balanced Acc: 0.8053763440860215
Sensitivity: 0.7666666666666667
Specificity: 0.8440860215053764
=====


=====
PRL
AUC: 0.8301824212271973
Balanced Acc: 0.7582089552238807
Sensitivity: 0.8
Specificity: 0.7164179104477612
=====


=====
PV
AUC: 0.9071555292726197
Balanced Acc: 0.7953873447664104
Sensitivity: 0.7368421052631579
Specificity: 0.8539325842696629
=====


=====
SPC
AUC: 0.7569444444444444
Balanced Acc: 0.6845238095238095
Sensitivity: 0.7916666666666666
Specificity: 0.5773809523809523
=====


=====
APC
AUC: 0.7385416666666667
Balanced